## For the training part refer to link https://www.kaggle.com/nayakroshan/pytorch-efficientnet-folds

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2 as cv
import glob

from tqdm.auto import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import albumentations
from albumentations.pytorch.transforms import ToTensorV2

import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm

import gc
gc.enable()

import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
class PetNet(nn.Module):
    def __init__(self, model_name, out_features, inp_channels, pretrained, num_dense):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, in_chans=inp_channels)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, 128)
        self.fc = nn.Sequential(
            nn.Linear(128 + num_dense, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, image, dense):
        embeddings = self.model(image)
        x = self.dropout(embeddings)
        x = torch.cat([x, dense], dim=1)
        output = self.fc(x)
        return output

In [ ]:
def test_fn(test_loader, model, device):
    model.eval()
    stream = tqdm(test_loader)
    final_outputs = []
    
    with torch.no_grad():
        for i, (image, dense) in enumerate(stream, start=1):
            image = image.to(device, non_blocking=True)
            dense = dense.to(device, non_blocking=True)
            output = model(image, dense)
            outputs = (torch.sigmoid(output).detach().cpu().numpy()*100).tolist()
            final_outputs.extend(outputs)
        
    return final_outputs

In [ ]:
def test_transform_object(DIM = 384):
    return albumentations.Compose(
        [
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(p=1.0)
        ]
    )

In [ ]:
class PetTestset(Dataset):
    
    def __init__(self, image_paths, dense_features, transform=None):
        self.image_paths = image_paths
        self.dense_feats = dense_features
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        #read the image using the path.
        img = cv.imread(self.image_paths[index], 1)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        img = cv.resize(img, (224, 224), interpolation = cv.INTER_AREA)
#         img = np.transpose(img, (2, 0, 1))
#         img = torch.Tensor(img)
        
        if self.transform is not None:
            img = self.transform(image=img)['image']
            
        img = img.float()
            
        #get the dense features.
        dense = self.dense_feats[index, :]
        
        return (img, dense)

In [ ]:
predicted_labels = None
models_dir = '../input/modelv7'
model_params = {
    'model_name' : 'swin_small_patch4_window7_224',
    'out_features' : 1,
    'inp_channels' : 3,
    'num_dense' : 12,
    'pretrained' : False
}

def get_testset(df, images):
    ids = list(df['Id'])
    image_paths = [os.path.join(images, idx + '.jpg') for idx in ids]
    df.drop(['Id'], inplace=True, axis=1)
    dense_feats = df.values
    test_transform = test_transform_object()
    return PetTestset(image_paths, dense_feats, test_transform)

outputs = None
for model_name in glob.glob(models_dir + '/*.pth'):
    model = PetNet(**model_params)
    model.load_state_dict(torch.load(model_name))
    model = model.to(device)
    
    test_images = '../input/petfinder-pawpularity-score/test'
    test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
    testset = get_testset(test_df, test_images)
    test_loader = DataLoader(testset, batch_size=16, shuffle=False)
    
    if outputs is None:
        outputs = test_fn(test_loader, model, device)
    else:
        temp = test_fn(test_loader, model, device)
        for i in range(len(temp)):
            outputs[i].append(temp[i][0])
            
for i in range(len(outputs)):
    outputs[i] = [sum(outputs[i]) / (len(glob.glob(models_dir + '/*.pth')))]

In [ ]:
sub_csv = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
for i in range(len(outputs)):
    sub_csv.loc[i, 'Pawpularity'] = outputs[i][0]

In [ ]:
sub_csv.to_csv('submission.csv', index=False)

In [ ]:
sub_csv.head()